In [ ]:
import sys
sys.path.append("/Users/admin/Documents/github/UGFAFAFA/code/")
import logging

In [ ]:
def log(msf):
    logging.debug(msf)

In [4]:
#  引入文件
from Tusharedata.daily import load
import pandas as pd

# 加载数据
ma = 30
df = load(code="300059.SZ")
df = df[df["date"] > "20210101"]
df.index= pd.to_datetime(df["date"])
from Analyse.haikui import signal,strategy
signal(df,ma)

# 加载回测系统
from rolltrader.cerebro import Cerebro
cerebro = Cerebro()
cerebro.strategy = strategy
cerebro.data = df
cerebro.log = log
edf = cerebro.run()

df = df.join(edf)
result = df[["date","cash","position"]]
result




DEBUG:root:>>>>> load DataCache
2021-05-16 13:43:54,276 INFO sqlalchemy.engine.Engine SELECT data_cache.id AS data_cache_id, data_cache."key" AS data_cache_key, data_cache.value AS data_cache_value 
FROM data_cache 
WHERE data_cache."key" = ?
 LIMIT ? OFFSET ?
INFO:sqlalchemy.engine.Engine:SELECT data_cache.id AS data_cache_id, data_cache."key" AS data_cache_key, data_cache.value AS data_cache_value 
FROM data_cache 
WHERE data_cache."key" = ?
 LIMIT ? OFFSET ?
2021-05-16 13:43:54,278 INFO sqlalchemy.engine.Engine [cached since 67.42s ago] ('daily.300059.sz', 1, 0)
INFO:sqlalchemy.engine.Engine:[cached since 67.42s ago] ('daily.300059.sz', 1, 0)
DEBUG:root:>>>>> load DataCache end
DEBUG:root:>>>>> load return data
INFO:root:mas BEGIN
INFO:root:mas  END
DEBUG:root:{'bcomm': 0, 'scomm': 0, 'berror': None, 'serror': None, 'bcount': 0, 'bprice': 0, 'scount': 0, 'sprice': 0, 'cash': 10000, 'position': 0}
DEBUG:root:{'bcomm': 0, 'scomm': 0, 'berror': None, 'serror': None, 'bcount': 0, 'bpric